In [1]:
import xarray as xr
import numpy as np
import sys
import time
from scipy.stats import weibull_min
import dask.distributed as dd

root_dir = '/media/harish/SSD_4TB/EU_SCORES_project'
scripts_dir = f'{root_dir}/scripts'
sys.path.append(scripts_dir)

from data_processing.libraries import mean_statistics, std_statistics

In [9]:
client.close()
cluster.close()

In [2]:
import dask.distributed as dd
cluster = dd.LocalCluster(n_workers=24, dashboard_address=':22722')
client = dd.Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:22722/status,
Dashboard: http://127.0.0.1:22722/status,Workers: 24
Total threads: 96,Total memory: 187.55 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40101,Workers: 24
Dashboard: http://127.0.0.1:22722/status,Total threads: 96
Started: Just now,Total memory: 187.55 GiB
Comm: tcp://127.0.0.1:41611,Total threads: 4
Dashboard: http://127.0.0.1:34199/status,Memory: 7.81 GiB
Nanny: tcp://127.0.0.1:35535,


In [3]:
root_dir = '/media/harish/SSD_4TB/EU_SCORES_project'
run = 'New_runs' #sys.argv[1]
case = 'Germany_coast' #sys.argv[2]
file_name = 'ws_10' #sys.argv[3]
variable = 'ws' #sys.argv[4]
chunks={"Time": -1,"south_north": 8,"west_east": 8}
run_dir=f'{root_dir}/WRFV4.4/EU_SCORES/{run}/{case}/Postprocessed/variablewise_files'
target_dir=f'{root_dir}/WRFV4.4/EU_SCORES/{run}/{case}/Postprocessed/statistics_files/{file_name}'
ds = xr.open_dataset(f'{run_dir}/{file_name}.nc',chunks=chunks)[variable]
data = ds.isel(Time=slice(None,-1))
data

<xarray.DataArray 'ws' (Time: 271728, south_north: 63, west_east: 63)>
dask.array<getitem, shape=(271728, 63, 63), dtype=float32, chunksize=(271728, 8, 8), chunktype=numpy.ndarray>
Coordinates:
  * Time     (Time) datetime64[ns] 1990-01-01 ... 2020-12-31T23:00:00
    XLONG    (south_north, west_east) float32 dask.array<chunksize=(8, 8), meta=np.ndarray>
    XLAT     (south_north, west_east) float32 dask.array<chunksize=(8, 8), meta=np.ndarray>
Dimensions without coordinates: south_north, west_east

In [18]:
data.groupby('Time.hour').quantile(0.05, dim='Time', method='inverted_cdf').compute()

<xarray.DataArray 'ws' (hour: 24, south_north: 63, west_east: 63)>
array([[[2.4336307, 2.4163816, 2.4099295, ..., 2.3917274, 2.3949392,
         2.4086883],
        [2.410416 , 2.3743556, 2.3802485, ..., 2.3628244, 2.3758664,
         2.4018338],
        [2.4058483, 2.367615 , 2.358945 , ..., 2.3373928, 2.3652062,
         2.3962126],
        ...,
        [2.4405954, 2.4034019, 2.3878782, ..., 2.4057727, 2.4256182,
         2.453462 ],
        [2.450168 , 2.4177887, 2.4107196, ..., 2.4236636, 2.432696 ,
         2.4501112],
        [2.4594102, 2.4608831, 2.4607053, ..., 2.4550638, 2.450853 ,
         2.4545913]],

       [[2.402017 , 2.380468 , 2.3756974, ..., 2.3677773, 2.3722942,
         2.3892887],
        [2.3880384, 2.3505642, 2.3402658, ..., 2.3416371, 2.3522422,
         2.3815155],
        [2.3899503, 2.3432455, 2.3328545, ..., 2.3223374, 2.3573709,
         2.3831172],
...
        [2.3320303, 2.308876 , 2.2915561, ..., 2.2632833, 2.284667 ,
         2.316424 ],
        [2.343036 , 2.3379147, 2.3242292, ..., 2.2857478, 2.3034313,
         2.3176024],
        [2.3543677, 2.3563209, 2.3588836, ..., 2.3142369, 2.3001378,
         2.3145442]],

       [[2.3506856, 2.3394446, 2.3370101, ..., 2.3259065, 2.3285348,
         2.3510525],
        [2.3412673, 2.3047895, 2.2874546, ..., 2.2897553, 2.3109221,
         2.3294036],
        [2.3366814, 2.2927725, 2.254378 , ..., 2.286232 , 2.3028417,
         2.3326657],
        ...,
        [2.3871415, 2.3421972, 2.3262796, ..., 2.3389523, 2.3673472,
         2.3996742],
        [2.3914676, 2.3593342, 2.3624325, ..., 2.3748205, 2.3855784,
         2.3993592],
        [2.3929093, 2.3899405, 2.3979456, ..., 2.402867 , 2.4131021,
         2.4191153]]], dtype=float32)
Coordinates:
    XLONG     (south_north, west_east) float32 7.657 7.665 7.672 ... 8.137 8.145
    XLAT      (south_north, west_east) float32 54.06 54.06 54.06 ... 54.34 54.34
    quantile  float64 0.05
  * hour      (hour) int64 0 1 2 3 4 5 6 7 8 9 ... 14 15 16 17 18 19 20 21 22 23
Dimensions without coordinates: south_north, west_east